In [1]:

import cv2
import numpy as np
from matplotlib import pyplot as plt 
from numpy import asarray
from PIL import Image
import os
import tensorflow as tf
import tensorflow.keras as k
from tensorflow.keras.models import Sequential 
import pickle
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from datetime import datetime
import csv

In [22]:

# List of folders
folders = ["C:/Users/rodea/OneDrive/Desktop/Dataset/J004","C:/Users/rodea/OneDrive/Desktop/Dataset/J011","C:/Users/rodea/OneDrive/Desktop/Dataset/J012","C:/Users/rodea/OneDrive/Desktop/Dataset/J015",
           "C:/Users/rodea/OneDrive/Desktop/Dataset/J016","C:/Users/rodea/OneDrive/Desktop/Dataset/J021","C:/Users/rodea/OneDrive/Desktop/Dataset/J023","C:/Users/rodea/OneDrive/Desktop/Dataset/J024",
           "C:/Users/rodea/OneDrive/Desktop/Dataset/J025","C:/Users/rodea/OneDrive/Desktop/Dataset/J031","C:/Users/rodea/OneDrive/Desktop/Dataset/J037","C:/Users/rodea/OneDrive/Desktop/Dataset/J056",
           "C:/Users/rodea/OneDrive/Desktop/Dataset/J058","C:/Users/rodea/OneDrive/Desktop/Dataset/J065","C:/Users/rodea/OneDrive/Desktop/Dataset/J066","C:/Users/rodea/OneDrive/Desktop/Dataset/J069",
           "C:/Users/rodea/OneDrive/Desktop/Dataset/J074"]
        
          
          

# An empty list to store processed images
train_images = []

# Iterate through each folder
for folder in folders:
    # List all files in the folder
    files = os.listdir(folder)

    # Iterate through each file
    for file in files:
        # Get the full file path
        file_path = os.path.join(folder, file)

        # Read the image
        img = cv2.imread(file_path)

    
        processed_img = cv2.resize(img, (200,400))  
        processed_img = processed_img / 255.0  

        # Add processed image to the list
        train_images.append(processed_img)

concatenated_images = np.array(train_images)


print("Shape of concatenated images:", concatenated_images.shape)


Shape of concatenated images: (850, 400, 200, 3)


In [23]:
num_classes = 17

labels = y = np.repeat([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],50)

#one-hot encoding
y = tf.keras.utils.to_categorical(labels, num_classes)
print(y[48:52])

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(400, 200, 3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(17, activation='softmax')  
])




In [12]:
# Compile the model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [13]:
model.fit(concatenated_images,y,epochs =10)

Epoch 1/10
27/27 [==============================] - 107s 4s/step - loss: 1.6997 - accuracy: 0.6165
Epoch 2/10
27/27 [==============================] - 107s 4s/step - loss: 0.0019 - accuracy: 0.9988
Epoch 3/10
27/27 [==============================] - 121s 4s/step - loss: 1.2534e-05 - accuracy: 1.0000
Epoch 4/10
27/27 [==============================] - 130s 5s/step - loss: 5.8466e-06 - accuracy: 1.0000
Epoch 5/10
27/27 [==============================] - 154s 6s/step - loss: 4.0557e-06 - accuracy: 1.0000
Epoch 6/10
27/27 [==============================] - 116s 4s/step - loss: 3.1687e-06 - accuracy: 1.0000
Epoch 7/10
27/27 [==============================] - 111s 4s/step - loss: 2.5521e-06 - accuracy: 1.0000
Epoch 8/10
27/27 [==============================] - 132s 5s/step - loss: 2.1029e-06 - accuracy: 1.0000
Epoch 9/10
27/27 [==============================] - 152s 6s/step - loss: 1.7246e-06 - accuracy: 1.0000
Epoch 10/10
27/27 [==============================] - 133s 5s/step - loss: 1.4432e

In [14]:
##storing the model
model.save('simple_cnn.h5')

In [2]:
##lIVE USING CAMERA

import cv2
import pickle
import numpy as np
import os
from datetime import datetime
from keras.models import load_model
import csv

model = load_model('simple_cnn.h5')


def preprocess_image(frame):
    
    processed_image = frame / 255.0  
    return processed_image


attendance = True
cap = cv2.VideoCapture(0)


face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


with open('attendance_log.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Date', 'Time', 'Prediction'])

logged_predictions = set() 

def log_attendance(predicted_class):
    if predicted_class not in logged_predictions:  
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        current_date = now.strftime("%d/%m/%Y")

        with open('attendance_log.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([current_date, current_time, predicted_class])

        logged_predictions.add(predicted_class)  

while attendance:
    ret, frame = cap.read()
    
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  ##convert into gray scale

    
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30)) ##for face  detection

    for (x, y, w, h) in faces:
        padding = 50
        face_center_x = x + w // 2
        face_center_y = y + h // 2
        roi = frame[max(0, face_center_y - h // 2 - padding):min(frame.shape[0], face_center_y + h // 2 + padding),   ##region of interest
                    max(0, face_center_x - w // 2 - padding):min(frame.shape[1], face_center_x + w // 2 + padding)]

        resized_face = cv2.resize(roi, (200, 400))

        
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2) ##rectangle frame

        processed_image = resized_face / 255.0  
        prediction = model.predict(np.array([processed_image]))

        class_labels = ['J004-Jay Ajmera','J011-Savali Chavan','J012-Snehee Cheeda','J015 -Farid Damania','J016-Aditya Das','J021- Heet Dhandukia','J023-Anish Gharat','J024-Suchetan Ghosh',
                        'J025-Monish Gosar','J031- Naiktik Jain','J037-Rudra Joshi','J056-Atharva Rode','J058-Jash Shah','J065- Kallind Soni','J066- Naman Upadhyay','J069- Ismail Wangde','J074- Mihir Shah']
        predicted_class = class_labels[np.argmax(prediction)]

        log_attendance(predicted_class)

        annotation = predicted_class
        cv2.putText(frame, annotation, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    frame_height, frame_width, _ = frame.shape
    frame = cv2.resize(frame, (int(frame_width * (800 / frame_height)), 400))   # frame to match the height of the resized face (2:1 ratio)
  

    cv2.imshow('Face Detection and Prediction', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break





1/1 [==============================] - 0s 114ms/step
